### Import necessary libraries, set options

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import patsy
import re
import seaborn as sns
import statsmodels.api as sm
import warnings

from statsmodels.formula.api import glm

pd.set_option('display.max_columns', 125)
warnings.filterwarnings("ignore")

### Read in pickled datasets

In [ ]:
path_to_diss = os.path.join("path/to/dissertation/chapter")

path_to_data = os.path.join(path_to_diss, "path/to/data/processed-data")

##### Between-individual

In [ ]:
data_final_coop = pd.read_pickle(path_to_data + "/pkl/data_final_coop.pkl")
print(len(data_final_coop))
data_final_coop.head()

In [ ]:
data_pid_version = pd.read_pickle(path_to_data + "/pkl/data_pid_version.pkl")
print(len(data_pid_version))
data_pid_version.head()

##### Within-individual

In [ ]:
data_within_coop = pd.read_pickle(path_to_data + "/pkl/data_within_coop.pkl")
print(len(data_within_coop))
data_within_coop.head()

In [ ]:
data_within_pid = pd.read_pickle(path_to_data + "/pkl/data_within_pid.pkl")
print(len(data_within_pid))
data_within_pid.head()

### Generate within-subjects interaction term

In [ ]:
data_within_pid['game1_int'] = data_within_pid['earned_v1'] * data_within_pid['equal_v1']
data_within_pid['game2_int'] = data_within_pid['earned_v2'] * data_within_pid['equal_v2']
data_within_pid['delta_int'] = data_within_pid['game2_int'] - data_within_pid['game1_int']

In [ ]:
data_within_coop['game1_int'] = data_within_coop['earned_v1'] * data_within_coop['equal_v1']
data_within_coop['game2_int'] = data_within_coop['earned_v2'] * data_within_coop['equal_v2']
data_within_coop['delta_int'] = data_within_coop['game2_int'] - data_within_coop['game1_int']

### Results

##### Fairness perceptions model

In [ ]:
y, X = patsy.dmatrices(
    'delta_f_score ~ delta_earned + delta_equal + delta_int', 
    # 'delta_f_score ~ delta_earned * delta_equal'
    data_within_pid,
    return_type = 'dataframe'
)

session_c = data_within_pid[(data_within_pid['delta_f_score'] == data_within_pid['delta_f_score']) &
                            (data_within_pid['delta_earned']  == data_within_pid['delta_earned'])  &
                            (data_within_pid['delta_equal']   == data_within_pid['delta_equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

##### Cooperation patterns model

In [ ]:
y, X = patsy.dmatrices(
    'delta_coopChoice ~ delta_earned + delta_equal + delta_int + C(round)', 
    # 'delta_coopChoice ~ delta_earned * delta_equal + C(round)'
    data_within_coop,
    return_type = 'dataframe'
)

session_c = data_within_coop[(data_within_coop['round']            == data_within_coop['round'])            &
                             (data_within_coop['delta_coopChoice'] == data_within_coop['delta_coopChoice']) &
                             (data_within_coop['delta_earned']     == data_within_coop['delta_earned'])     &
                             (data_within_coop['delta_equal']      == data_within_coop['delta_equal'])]['session_no']

pid_c     = data_within_coop[(data_within_coop['round']            == data_within_coop['round'])            &
                             (data_within_coop['delta_coopChoice'] == data_within_coop['delta_coopChoice']) &
                             (data_within_coop['delta_earned']     == data_within_coop['delta_earned'])     &
                             (data_within_coop['delta_equal']      == data_within_coop['delta_equal'])]['pid']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Instrumental variable model

In [ ]:
y, X = patsy.dmatrices(
    'delta_f_score ~ delta_earned + delta_equal + delta_int', 
    # 'delta_f_score ~ delta_earned * delta_equal'
    data_within_pid,
    return_type = 'dataframe'
)

session_c = data_within_pid[(data_within_pid['delta_f_score'] == data_within_pid['delta_f_score']) &
                            (data_within_pid['delta_earned']  == data_within_pid['delta_earned'])  &
                            (data_within_pid['delta_equal']   == data_within_pid['delta_equal'])]['session_no']

ols = sm.OLS(y, X)
ols_res = ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]})

data_within_pid['ols_preds'] = ols_res.predict(X)

print(data_within_pid.shape)
data_within_pid.head()

In [ ]:
print(data_within_coop.shape)
data_within_coop.head()

In [ ]:
df = data_within_coop.merge(data_within_pid[['session_no', 'pid', 'ols_preds']], 
                            on = ['session_no', 'pid'])

print(df.shape)
df.head()

In [ ]:
y, X = patsy.dmatrices('delta_coopChoice ~ ols_preds + C(round)', df, return_type = 'dataframe')

session_c = df[(df['round']            == df['round'])            &
               (df['delta_coopChoice'] == df['delta_coopChoice']) &
               (df['ols_preds']        == df['ols_preds'])]['session_no']

pid_c     = df[(df['round']            == df['round'])            &
               (df['delta_coopChoice'] == df['delta_coopChoice']) &
               (df['ols_preds']        == df['ols_preds'])]['pid']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Figure: heatmaps

In [ ]:
def get_condition_pairs(row):
    if   row.condition_v1 == 'ru' and row.condition_v2 == 'ru': return 'RU to RU'
    elif row.condition_v1 == 'ru' and row.condition_v2 == 'ee': return 'RU to EE'
    elif row.condition_v1 == 'ru' and row.condition_v2 == 're': return 'RU to RE'
    elif row.condition_v1 == 'ru' and row.condition_v2 == 'eu': return 'RU to EU'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 'ru': return 'EE to RU'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 'ee': return 'EE to EE'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 're': return 'EE to RE'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 'eu': return 'EE to EU'
    elif row.condition_v1 == 're' and row.condition_v2 == 'ru': return 'RE to RU'
    elif row.condition_v1 == 're' and row.condition_v2 == 'ee': return 'RE to EE'
    elif row.condition_v1 == 're' and row.condition_v2 == 're': return 'RE to RE'
    elif row.condition_v1 == 're' and row.condition_v2 == 'eu': return 'RE to EU'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 'ru': return 'EU to RU'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 'ee': return 'EU to EE'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 're': return 'EU to RE'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 'eu': return 'EU to EU'
    
def get_change_in_f_score(row):
    return row.f_score_v2 - row.f_score_v1

def get_change_in_cooperation(row):
    return row.coopChoice_v2 - row.coopChoice_v1

In [ ]:
# Fairness model

temp = data_pid_version[['session_no', 'pid', 'version', 'condition', 'f_score']]

v1 = temp[temp['version'] == 1]
v2 = temp[temp['version'] == 2]

df = v1.merge(v2, on = ['session_no', 'pid'], suffixes = ["_v1", "_v2"])

df = df[['session_no', 'pid', 'condition_v1', 'condition_v2', 'f_score_v1', 'f_score_v2']]

df['condition_pair']    = df.apply(get_condition_pairs,   axis = 1)
df['change_in_f_score'] = df.apply(get_change_in_f_score, axis = 1)

#y, X = patsy.dmatrices('change_in_f_score ~ C(condition_pair) - 1', df, return_type = 'dataframe')

mod = glm("change_in_f_score ~ C(condition_pair) - 1", data = df)

session_c = df[(df['change_in_f_score'] == df['change_in_f_score']) &
               (df['condition_pair']    == df['condition_pair'])]['session_no']

#ols = sm.OLS(y, X)
#ols.fit(cov_type = 'cluster', cov_kwds = {'groups': [session_c]}).summary()

mod.fit_constrained(
    'C(condition_pair)[RU to RU] + C(condition_pair)[RU to EE]  + \
     C(condition_pair)[RU to RE] + C(condition_pair)[RU to EU]  + \
     C(condition_pair)[EE to RU] + C(condition_pair)[EE to EE]  + \
     C(condition_pair)[EE to RE] + C(condition_pair)[EE to EU]  + \
     C(condition_pair)[RE to RU] + C(condition_pair)[RE to EE] + \
     C(condition_pair)[RE to RE] + C(condition_pair)[RE to EU] + \
     C(condition_pair)[EU to RU] + C(condition_pair)[EU to EE] + \
     C(condition_pair)[EU to RE] + C(condition_pair)[EU to EU]',
    cov_type = 'cluster', cov_kwds = {'groups': [session_c]}).summary()

In [ ]:
# Cooperation model

temp = data_final_coop[['session_no', 'pid', 'version', 'condition', 'round', 'coopChoice']]

v1 = temp[temp['version'] == 1]
v2 = temp[temp['version'] == 2]

df = v1.merge(v2, on = ['session_no', 'pid', 'round'], suffixes = ["_v1", "_v2"])

df = df[['session_no', 'pid', 'round', 'condition_v1', 'condition_v2', 'coopChoice_v1', 'coopChoice_v2']]

df['condition_pair']        = df.apply(get_condition_pairs,       axis = 1)
df['change_in_cooperation'] = df.apply(get_change_in_cooperation, axis = 1)

mod = glm("change_in_cooperation ~ C(condition_pair) + C(round) - 1", data = df)

session_c = df[(df['change_in_cooperation'] == df['change_in_cooperation']) &
               (df['condition_pair']        == df['condition_pair'])        &
               (df['round']                 == df['round'])]['session_no']

pid_c     = df[(df['change_in_cooperation'] == df['change_in_cooperation']) &
               (df['condition_pair']        == df['condition_pair'])        &
               (df['round']                 == df['round'])]['pid']

mod.fit_constrained(
    'C(condition_pair)[RU to RU] + C(condition_pair)[RU to EE]  + \
     C(condition_pair)[RU to RE] + C(condition_pair)[RU to EU]  + \
     C(condition_pair)[EE to RU] + C(condition_pair)[EE to EE]  + \
     C(condition_pair)[EE to RE] + C(condition_pair)[EE to EU]  + \
     C(condition_pair)[RE to RU] + C(condition_pair)[RE to EE] + \
     C(condition_pair)[RE to RE] + C(condition_pair)[RE to EU] + \
     C(condition_pair)[EU to RU] + C(condition_pair)[EU to EE] + \
     C(condition_pair)[EU to RE] + C(condition_pair)[EU to EU]',
    cov_type = 'cluster', cov_kwds = {'groups': [session_c, pid_c]}).summary()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (12, 7.5))

#                   RU       EE       RE       EU
array = np.array([[ 0.3656,  0.1678,  0.4307,  1.4473],  # RU
                  [-0.2489, -0.0363,  0.1452,  1.3979],  # EE
                  [-0.4668, -0.4772,  0.4768,  0.6760],  # RE
                  [-1.3072, -1.1365, -1.0854, -0.3490]]) # EU

# annot = np.array([["p=0.058", "",        "",        "p<0.001"],
#                   ["",        "",        "",        "p<0.001"],
#                   ["",        "p=0.023", "p=0.029", "p=0.002"],
#                   ["p<0.001", "p<0.001", "p<0.001", ""       ]])

annot = np.array([["", "", "", ""],
                  ["", "", "", ""],
                  ["", "", "", ""],
                  ["", "", "", ""]])

sns.heatmap(array,
            vmin = -1.7,
            vmax =  1.7,
            annot = annot,
            fmt = '',
            cbar_kws={'label': 'Change in perceived fairness score',
                      'orientation': 'horizontal'},
            cmap = "RdBu_r",
            ax = ax[0])

ax[0].set_xticklabels(["RU", "EE", "RE", "EU"])
ax[0].set_yticklabels(["RU", "EE", "RE", "EU"], rotation = 0)
ax[0].set_xlabel("Second game")
ax[0].set_ylabel("First game")
ax[0].set_title("Fairness")

#                   RU       EE       RE       EU
array = np.array([[ 0.0332, -0.0404, -0.0230, -0.0192],  # RU
                  [ 0.0510, -0.0232, -0.0014, -0.0266],  # EE
                  [ 0.0103, -0.0888,  0.0175, -0.0174],  # RE
                  [ 0.0016,  0.0674,  0.0588,  0.0002]]) # EU

# #                   RU       EE       RE       EU
# array = np.array([[ 3.32, -4.04, -2.30, -1.92],  # RU
#                   [ 5.10, -2.32, -0.14, -2.66],  # EE
#                   [ 1.03, -8.88,  1.75, -1.74],  # RE
#                   [ 0.16,  6.74,  5.88,  0.02]]) # EU

# annot = np.array([["", "p=0.072", "",        ""],
#                   ["", "",        "",        ""],
#                   ["", "p=0.005", "",        ""],
#                   ["", "p=0.019", "p=0.100", ""]])

annot = np.array([["", "", "", ""],
                  ["", "", "", ""],
                  ["", "", "", ""],
                  ["", "", "", ""]])

sns.heatmap(array,
            vmin = -0.08, # -8
            vmax =  0.08, #  8
            annot = annot,
            fmt = '',
            cbar_kws={'label': 'Change in probability of cooperating', 
                      'orientation': 'horizontal'},
            cmap = "RdBu_r",
            ax = ax[1])
ax[1].set_xticklabels(["RU", "EE", "RE", "EU"])
ax[1].set_yticklabels(["RU", "EE", "RE", "EU"], rotation = 0)
ax[1].set_xlabel("Second game")
ax[1].set_ylabel("First game")
ax[1].set_title("Cooperation")

fig.savefig(os.path.join(path_to_diss, "paper/figures/figure2.png"), bbox_inches = 'tight', pad_inches = 0.25)

##### Figure: coefficient estimates 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 5), constrained_layout=True)

# Fairness
data = [['delta(Earned)',                 1.3859, 0.131*1.96],
        ['delta(Equal)',                  0.4816, 0.143*1.96],
        ['delta(Earned) x delta(Equal)', -1.7060, 0.175*1.96]]

data = pd.DataFrame(data, columns = ['Coefficient', 'Estimate', 'Standard Error']) 

sns.pointplot('Estimate', 'Coefficient', data=data, 
              dodge=True, join=False, ci=None, ax = ax[0], color='black', scale=0.75)

x_coords = []
y_coords = []
for point_pair in ax[0].collections:
    for x, y in point_pair.get_offsets():
        x_coords.append(x)
        y_coords.append(y)
        
ax[0].errorbar(x_coords, y_coords, xerr=data['Standard Error'], fmt=' ', zorder=-1, capsize=5, color='black')

ax[0].set_ylabel("")
ax[0].set_yticklabels([r"$\Delta$ Earned", r"$\Delta$ Equal", r"$\Delta$ Earned x $\Delta$ Equal"])
ax[0].axvline(0, linewidth=1, color='r', linestyle='--')
ax[0].set(xlim=(-2.5, 2.5))
ax[0].set_title("Fairness")
ax[0].xaxis.grid(True)
ax[0].yaxis.grid(True)
ax[0].xaxis.set_ticks(np.arange(-2, 3, 1))

# Cooperation
data = [['delta(Earned)',                -0.0415, 0.016*1.96],
        ['delta(Equal)',                 -0.0024, 0.016*1.96],
        ['delta(Earned) x delta(Equal)',  0.0147, 0.023*1.96]]

data = pd.DataFrame(data, columns = ['Coefficient', 'Estimate', 'Standard Error']) 

sns.pointplot('Estimate', 'Coefficient', data=data, 
              dodge=True, join=False, ci=None, ax = ax[1], color='black', scale=0.75)

x_coords = []
y_coords = []
for point_pair in ax[1].collections:
    for x, y in point_pair.get_offsets():
        x_coords.append(x)
        y_coords.append(y)
        
ax[1].errorbar(x_coords, y_coords, xerr=data['Standard Error'], fmt=' ', zorder=-1, capsize=5, color='black')

ax[1].set_ylabel("")
ax[1].set_yticklabels([r"$\Delta$ Earned", r"$\Delta$ Equal", r"$\Delta$ Earned x $\Delta$ Equal"])
ax[1].axvline(0, linewidth=1, color='r', linestyle='--')
ax[1].set(xlim=(-0.075, 0.075))
ax[1].set_title("Cooperation")
ax[1].xaxis.grid(True)
ax[1].yaxis.grid(True)
ax[1].xaxis.set_ticks(np.arange(-0.06, 0.09, 0.03))

sns.set_style("whitegrid")

fig.savefig(os.path.join(path_to_diss, "paper/figures/figure3-python.png"), bbox_inches = 'tight', pad_inches = 0.25)